# 💼 GenCareerPath 1.0 – Career Path Advisor

This notebook is the Capstone Project for the GenAI Intensive Course with Google.

**GenCareerPath 1.0** uses a Generative AI model (`gemini-2.0-flash`) to analyze a candidate's resume and suggest realistic or aspirational career paths based on the profile. It simulates the role of a career advisor enhanced by generative capabilities.

---

## 🎯 Objective

To demonstrate multiple GenAI capabilities in a practical, real-world application:
- Extract structured data from unstructured resume text
- Recommend personalized career directions
- Match user profiles to roles with explanation and skill development suggestions

---

## ✅ GenAI Capabilities Demonstrated in This Project

This project uses Gemini 2.0 to analyze a resume and generate career insights.

We demonstrate the following official GenAI capabilities required by the Capstone:

| Capability                                | Applied in this project                                          |
|-------------------------------------------|------------------------------------------------------------------|
| **Structured output / JSON mode**         | Resume is converted to structured fields using controlled output |
| **Few-shot prompting**                    | Prompts define and constrain the expected response structure     |
| **Document understanding**                | Model extracts information from a free-text resume               |
---

## 🔧 Technologies
- Google Generative AI SDK (`google-generativeai`)
- Gemini 2.0 Flash model (`models/gemini-2.0-flash`)
- Kaggle Secrets for secure API usage

## 🛠️ Step 0 - Setup

## 🔐 API Key Configuration (Required)

To run this notebook, you **must add your Google Generative AI API key** using the Kaggle Secrets manager.

---

### 📌 Steps:

1. Go to the **right sidebar** → click on **"Add-ons" → "Secrets"**
2. Create a new secret with the following:

- **Name**: `GOOGLE_API_KEY`  *(must match exactly)*
- **Value**: Your actual API key from Google Generative AI

---

> ⚠️ **If the API key is missing, the notebook will fail to connect to the model.**

---

### ✅ Referenced Code Snippet

```python
from kaggle_secrets import UserSecretsClient
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [1]:
!pip install -q --upgrade google-generativeai

from kaggle_secrets import UserSecretsClient
from IPython.display import display, HTML, Markdown
import google.generativeai as genai
import json

# Validate and load API key
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY.strip():
        raise ValueError("Empty key")
except Exception as e:
    display(HTML(f"""
        <p style='color:red; font-weight:bold; font-size:16px;'>
        ❌ ERROR: Google API key is missing or not set in Kaggle secrets.<br>
        Please create a secret named <code>GOOGLE_API_KEY</code> to proceed.
        </p>
    """))
    raise SystemExit("🛑 Halting execution due to missing API key.")

# Configure GenAI
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("models/gemini-2.0-flash")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00


## 📋 Step 1 - Paste Your Resume

Please paste your resume in the text area below **before running any code**.

If you leave it empty, the default sample resume will be used for demonstration purposes.

👉 Once you've pasted your resume, run the next cells to begin the analysis.

## 📚 Example Resumes

You can copy and paste any of the following resumes into the input field above to test the ResumeGenAI assistant.  
Each resume represents a different role and background.

---

### 🧠 GenAI Engineer

Generative AI Engineer with 5+ years experience in machine learning and NLP.
Worked with LLMs, prompt engineering, and fine-tuning techniques using TensorFlow and JAX.
Built internal RAG tools using vector databases and document embeddings.
Deployed multi-modal models for enterprise search, chatbot integration, and evaluation pipelines.
Familiar with Vertex AI, LangChain, and API-based orchestration for agent workflows.

---

### 🎨 UX Designer

Senior UX Designer with 8+ years experience in user-centered design.
Led end-to-end product design for fintech and e-commerce platforms.
Skilled in Figma, design systems, and user research methodologies.
Delivered responsive prototypes, A/B tests, and UX writing standards.
Passionate about accessibility, product strategy, and mentoring junior designers.

---

### 📊 Data Scientist

Data Scientist with 6+ years experience in predictive modeling and analytics.
Specialized in Python, pandas, scikit-learn, and PyTorch.
Built real-time dashboards, classification models, and business intelligence workflows.
Worked with stakeholders to translate data into product insights.
Strong communication, experimentation, and statistical modeling background.


In [2]:
from IPython.display import display, HTML
import ipywidgets as widgets

cv_input = widgets.Textarea(
    value="",
    placeholder="Paste your resume here...",
    description='Resume:',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
)

display(HTML("<h4>📄 ⚠️ You haven't pasted any resume yet. The default example will be used.</h4>"))
display(cv_input)

Textarea(value='', description='Resume:', layout=Layout(height='200px', width='100%'), placeholder='Paste your…

In [3]:
# Default resume (used only if input is empty)
sample_resume = """
Senior Software Engineer with 6+ years experience in backend development using Python, Go, and Java.
Led a team of 4 engineers. Built scalable APIs, integrated third-party services, and designed microservices architecture.
Experience with Kubernetes, Docker, GCP, PostgreSQL, Redis.
Interest in AI/ML and product management.
"""

# Use resume from input (cv_input is assumed to be defined in a previous cell)
resume = cv_input.value.strip()
if not resume:
    display(HTML("<p style='color:orange; font-weight:bold;'>⚠️ Resume input is empty. Using default sample resume.</p>"))
    resume = sample_resume

# STEP 1 – Extract structured profile
profile_prompt = f"""
You are a resume analyst.
Given the following resume, extract structured fields in JSON:
- years_experience: integer
- primary_stack: list of technologies or frameworks
- leadership_experience: boolean
- interests: list of areas
- career_level: entry / mid / senior

Resume:
{resume}
"""
profile = model.generate_content(profile_prompt).text
Markdown("**📄 Extracted Profile:**\n" + profile)

# STEP 2 – Suggest career paths
career_prompt = f"""
Given the profile:
{profile}

Suggest at least 5 realistic or aspirational career paths.
Return a list of job titles with one-line descriptions.
"""
career_suggestions = model.generate_content(career_prompt).text
Markdown("**🔎 Career Suggestions:**\n" + career_suggestions)

# STEP 3 – Match top roles
match_prompt = f"""
Based on this profile:
{profile}

And these career paths:
{career_suggestions}

Return a JSON array with 3 top recommended roles with:
- title
- why_match (1 sentence)
- suggested_next_skills (list)
"""
match = model.generate_content(match_prompt).text
Markdown("**🌐 Top Matches:**\n" + match)

**🌐 Top Matches:**
```json
[
  {
    "title": "Technical Lead",
    "why_match": "Combines the profile's strong technical skills in Python, Go, and cloud technologies with leadership experience to guide and mentor a team.",
    "suggested_next_skills": [
      "Project Management Methodologies (Agile, Scrum)",
      "Conflict Resolution",
      "Performance Management"
    ]
  },
  {
    "title": "Solutions Architect",
    "why_match": "Leverages the broad technical skillset across programming languages, cloud platforms, and databases to design and implement comprehensive solutions.",
    "suggested_next_skills": [
      "Enterprise Architecture Frameworks (TOGAF)",
      "Business Analysis",
      "Advanced Cloud Certifications (e.g., GCP Professional Architect)"
    ]
  },
  {
    "title": "Engineering Manager",
    "why_match": "Utilizes leadership experience and an interest in product management to manage a team, align technical strategy with business objectives, and focus on team development.",
    "suggested_next_skills": [
      "Budget Management",
      "Strategic Planning",
      "Advanced Product Management Techniques"
    ]
  }
]
```

## ✅ GenAI Capabilities Demonstrated

## ✅ Summary of GenAI Capabilities Used

| GenAI Capability                         | How it was applied                                                |
|------------------------------------------|-------------------------------------------------------------------|
| **Structured output / JSON mode**        | Model generates structured data (e.g., skills, experience level)  |
| **Few-shot prompting**                   | Prompts guide and shape responses with clear instructions         |
| **Document understanding**               | Unstructured CVs are analyzed to extract semantically rich fields |

These align directly with the required capabilities for the Capstone evaluation.
These capabilities are combined to simulate the work of an intelligent career advisor, powered by LLMs.